In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocess/3419_left.jpg
/kaggle/input/preprocess/4176_right.jpg
/kaggle/input/preprocess/679_left.jpg
/kaggle/input/preprocess/3370_left.jpg
/kaggle/input/preprocess/1255_right.jpg
/kaggle/input/preprocess/660_left.jpg
/kaggle/input/preprocess/484_right.jpg
/kaggle/input/preprocess/4221_right.jpg
/kaggle/input/preprocess/2396_left.jpg
/kaggle/input/preprocess/543_left.jpg
/kaggle/input/preprocess/3349_right.jpg
/kaggle/input/preprocess/3017_left.jpg
/kaggle/input/preprocess/1482_left.jpg
/kaggle/input/preprocess/497_left.jpg
/kaggle/input/preprocess/3142_right.jpg
/kaggle/input/preprocess/1133_right.jpg
/kaggle/input/preprocess/3099_left.jpg
/kaggle/input/preprocess/4597_left.jpg
/kaggle/input/preprocess/4640_left.jpg
/kaggle/input/preprocess/1036_right.jpg
/kaggle/input/preprocess/357_right.jpg
/kaggle/input/preprocess/2830_right.jpg
/kaggle/input/preprocess/908_right.jpg
/kaggle/input/preprocess/2412_left.jpg
/kaggle/input/preprocess/493_left.jpg
/kaggle/input/preproce

In [2]:
import pandas as pd
import numpy as np
import shutil
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [3]:
path_src = '/kaggle/input/preprocess'
path_dst = '/kaggle/working/ODIR_TrainingDataSet_Organisation/'           
path_dst_aug = '/kaggle/working/ODIR_TrainingDataSet_Augmented/'          
file_name_features = 'X'                          
file_name_target = 'y'                             
classes_columns = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

In [4]:
df = pd.read_csv('/kaggle/input/eyes-tabular/df_eyes_bueno.csv', index_col=0)

In [5]:
df

,Fundus,Diag,ID,A,C,D,G,H,M,N,O
0,0_left.jpg,C,0,False,True,False,False,False,False,False,False
1,0_right.jpg,N,0,False,False,False,False,False,False,True,False
2,1_left.jpg,N,1,False,False,False,False,False,False,True,False
3,1_right.jpg,N,1,False,False,False,False,False,False,True,False
4,2_right.jpg,D,2,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
6072,4688_left.jpg,D,4688,False,False,True,False,False,False,False,False
6073,4689_right.jpg,N,4689,False,False,False,False,False,False,True,False
6074,4689_left.jpg,D,4689,False,False,True,False,False,False,False,False
6075,4690_left.jpg,D,4690,False,False,True,False,False,False,False,False


In [6]:
# Create the path if it doesn´t exist
if ~os.path.exists(path_dst):
    os.makedirs(path_dst)

In [7]:
# Copia de archivos de imágenes y organización de imágenes por repertorio
for classe in classes_columns:
    dst = path_dst+classe+'/'
    if ~os.path.exists(dst):
        os.makedirs(dst, exist_ok=True)
    lst= df[df.Diag==classe].Fundus.tolist()
    print(f'Copy class {classe} in progress')
    for file in lst:
        src = os.path.join(path_src,file)
        shutil.copy(src, dst)

Copy class N in progress
Copy class D in progress
Copy class G in progress
Copy class C in progress
Copy class A in progress
Copy class H in progress
Copy class M in progress
Copy class O in progress


In [8]:
# Create the path if it doesn´t exist
if ~os.path.exists(path_dst_aug):
    os.makedirs(path_dst_aug)

In [9]:
#Creation of the image generator with specific changes
datagen = ImageDataGenerator(
    data_format='channels_last',
    rotation_range=40,
    height_shift_range=0.3,
    width_shift_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.4,0.9],
    #horizontal_flip=True,
    #vertical_flip=True,
    fill_mode='constant')

In [11]:
# Creation of additional images for each class in order to equilibrate the classes
for classe in classes_columns:

    if ~os.path.exists(path_dst_aug+'/'+classe):
        os.makedirs(path_dst_aug+'/'+classe, exist_ok=True)
    list_file = os.listdir(path_dst+'/'+classe)
    n = len(list_file)
    k = round(2816/n)
    print(f'classe : {classe}  initial quantity image:  {n}   Ratio: {k}  => Final quantity image : {n*k}')

    for file in list_file:
        file_image = path_dst+'/'+classe+'/'+file
        #print('file_image : ' , file_image)

        image = cv2.imread(file_image, cv2.IMREAD_COLOR)
        #image = cv2.resize(image, (224,224))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.expand_dims(image,0)
        datagen.fit(image)
        for x, val in zip(datagen.flow(image,
                                       save_to_dir=os.path.join(path_dst_aug,classe),
                                       save_prefix=file[:-4],
                                       save_format='png'),
                                        range(k-1)) :
            x_tmp = x
            val_tmp = val

classe : N  initial quantity image:  2816   Ratio: 1  => Final quantity image : 2816
classe : D  initial quantity image:  1524   Ratio: 2  => Final quantity image : 3048
classe : G  initial quantity image:  242   Ratio: 12  => Final quantity image : 2904
classe : C  initial quantity image:  252   Ratio: 11  => Final quantity image : 2772
classe : H  initial quantity image:  110   Ratio: 26  => Final quantity image : 2860
classe : M  initial quantity image:  244   Ratio: 12  => Final quantity image : 2928
classe : O  initial quantity image:  648   Ratio: 4  => Final quantity image : 2592


In [ ]:
# Convert it to a zip file
import shutil
shutil.make_archive('augmented_images', 'zip', '/kaggle/working/ODIR_TrainingDataSet_Augmented')

In [ ]:
from IPython.display import FileLink
FileLink(r'augmented_images.zip')